# The Determinants of Small Shareholder Victory

In [8]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [9]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [10]:
%%stata

import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear

* Parse date
gen day = date(date, "YMD")
format day %td
gen month = month(day)
gen quarter = quarter(day)

gen year = year(day)
encode gecko_id, gen(token)
encode space,    gen(dao)

* Dependent variables
local wins non_whale_victory_vn non_whale_victory_vp non_whale_victory_vp_vn

* Independent variables
local proposal_char multi_choices weighted delegation
local delegate_char whale_from_delegation_rate non_whale_from_delegation_rate ///
                    whale_to_delegation_rate non_whale_to_delegation_rate
local vote_char whale_vs_hhi non_whale_vs_hhi whale_vn_hhi non_whale_vn_hhi 
local user_char non_whale_user
local discussion concensus professionalism objectiveness data_intensity hhi_post_number

* Label variables
label var non_whale_victory_vn "\${\it Small Win}^{vn}_{i,t}\$"
label var non_whale_victory_vp "\${\it Small Win}^{vp}_{i,t}\$"
label var non_whale_victory_vp_vn "\${\it Small Win}^{vp,vn}_{i,t}\$"
label var non_whale_user "\${\it User}_{i,t}^{\textit{Small}}\$"
label var weighted "\${\it Weighted}_{i,t}\$"
label var quorum "\${\it Quorum}_{i,t}\$"
label var multi_choices "\${\it Multiple Choices}_{i,t}\$"
label var delegation "\${\it Delegation}_{i,t}\$"
label var professionalism  "\${\it Professionalism}_{i,t}\$"
label var objectiveness    "\${\it Objectiveness}_{i,t}\$"
label var concensus       "\${\it Concensus}_{i,t}\$"
label var data_intensity   "\${\it Data Intensity}_{i,t}\$"
label var hhi_post_number  "\${\it HHI}_{i,t}^{\# of Post}\$"
label var non_whale_from_delegation_rate "\$\textit{Delegator}_{i,t}^{\textit{Small}}\$"
label var whale_from_delegation_rate "\$\textit{Delegator}_{i,t}^{\textit{Whale}}\$"
label var non_whale_to_delegation_rate "\$\textit{Delegatee}_{i,t}^{\textit{Small}}\$"
label var whale_to_delegation_rate "\$\textit{Delegatee}_{i,t}^{\textit{Whale}}\$"
label var non_whale_vs_hhi "\$\textit{HHI}_{i,t}^{\textit{Small,vs}}\$"
label var whale_vs_hhi "\$\textit{HHI}_{i,t}^{\textit{Whale,vs}}\$"
label var non_whale_vn_hhi "\$\textit{HHI}_{i,t}^{\textit{Small,vn}}\$"
label var whale_vn_hhi "\$\textit{HHI}_{i,t}^{\textit{Whale,vn}}\$"


. 
. import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear
(encoding automatically selected: ISO-8859-1)
(57 vars, 2,841 obs)

. 
. * Parse date
. gen day = date(date, "YMD")

. format day %td

. gen month = month(day)

. gen quarter = quarter(day)

. 
. gen year = year(day)

. encode gecko_id, gen(token)

. encode space,    gen(dao)

. 
. * Dependent variables
. local wins non_whale_victory_vn non_whale_victory_vp non_whale_victory_vp_vn

. 
. * Independent variables
. local proposal_char multi_choices weighted delegation

. local delegate_char whale_from_delegation_rate non_whale_from_delegation_rate
>  ///
>                     whale_to_delegation_rate non_whale_to_delegation_rate

. local vote_char whale_vs_hhi non_whale_vs_hhi whale_vn_hhi non_whale_vn_hhi 

. local user_char non_whale_user

. local discussion concensus professionalism objectiveness data_intensity hhi_p
> ost_number

. 
. * Label variables
. label var non_whale_victory_vn "\${\it Small Win}^{vn}_{i,

## Summary Statistics

In [11]:
%%stata

******************************************************
* SUMMARY STATISTICS (before regression)
******************************************************
* Summarize all key variables
summarize `wins' `proposal_char' `delegate_char' `vote_char', detail

* Export a nice LaTeX summary table with mean, SD, p10, median, p90
estpost summarize `wins' `proposal_char' `delegate_char' `vote_char' `user_char', detail
esttab using "$TABLES/summary_stats.tex", replace ///
    cells("count(fmt(0)) mean(fmt(3)) sd(fmt(3)) p10(fmt(3)) p50(fmt(3)) p90(fmt(3))") ///
    collabels("Number of Obs." "Mean" "Std. Dev." "P10" "Median" "P90") ///
    nonumber nomtitles noobs label ///
    alignment(D{.}{.}{-1}) booktabs ///
    compress nofloat nostar ///
    substitute("\_" "_")                     ///
    title("Summary Statistics of Variables") ///
    prehead("\begin{tabular}{l*{6}{c}}\toprule") ///
    posthead("\hline") ///
    postfoot("\bottomrule\end{tabular}")


. 
. ******************************************************
. * SUMMARY STATISTICS (before regression)
. ******************************************************
. * Summarize all key variables
. summarize `wins' `proposal_char' `delegate_char' `vote_char', detail

                ${\it Small Win}^{vn}_{i,t}$
-------------------------------------------------------------
      Percentiles      Smallest
 1%            0              0
 5%            0              0
10%            0              0       Obs               2,681
25%            0              0       Sum of wgt.       2,681

50%            0                      Mean           .0675121
                        Largest       Std. dev.      .2509536
75%            0              1
90%            0              1       Variance       .0629777
95%            1              1       Skewness       3.447398
99%            1              1       Kurtosis       12.88455

                ${\it Small Win}^{vp}_{i,t}$
-------------------

## Proposal Characteristics

In [12]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

foreach y of local wins {

    * VIF test for multicollinearity
    qui reg `y' `user_char' `proposal_char'
    estat vif

    * Logit regression with SEs clustered by DAO
    logit `y' `proposal_char', vce(cluster dao)
    eststo `y'_p

    * Logit regression with SEs clustered by DAO
    logit `y' `user_char', vce(cluster dao)
    eststo `y'_u

    * Logit regression with SEs clustered by DAO
    logit `y' `user_char' `proposal_char', vce(cluster dao)
    eststo `y'_p_u

}

* Export LaTeX table
esttab                                                     ///
    non_whale_victory_vn_u                                 ///
    non_whale_victory_vn_p                                 ///
    non_whale_victory_vn_p_u                              ///
    non_whale_victory_vp_u                                 ///
    non_whale_victory_vp_p                                 ///
    non_whale_victory_vp_p_u                              ///
    non_whale_victory_vp_vn_u                              ///
    non_whale_victory_vp_vn_p                              ///
    non_whale_victory_vp_vn_p_u                           ///
    using "$TABLES/reg_logit_win_proposal.tex", replace    ///
    se star(* 0.10 ** 0.05 *** 0.01)                       ///
    b(%9.3f) se(%9.2f) label nogaps nonotes booktabs       ///
    noomitted eqlabels(none) nomtitles                     ///
    mgroups("\${\it Small Win}^{vn}_{i,t}\$"              ///
            "\${\it Small Win}^{vp}_{i,t}\$"              ///
            "\${\it Small Win}^{vp,vn}_{i,t}\$",          ///
                span                                      ///
                pattern(1 0 0 1 0 0 1 0 0)                      ///
                prefix(\multicolumn{@span}{c}{)           ///
                suffix(})                                 ///
                erepeat(\cmidrule(lr){@span}) )           ///
    substitute("\_" "_")                                   ///
    stats(N r2_p,                                          ///
        fmt(%9.0fc %9.3f)                                  ///
        labels("Observations" "Pseudo R²"))


. 
. ******************************************************
. * LOGISTIC REGRESSIONS
. ******************************************************
. 
. eststo clear

. 
. foreach y of local wins {
  2. 
.     * VIF test for multicollinearity
.     qui reg `y' `user_char' `proposal_char'
  3.     estat vif
  4. 
.     * Logit regression with SEs clustered by DAO
.     logit `y' `proposal_char', vce(cluster dao)
  5.     eststo `y'_p
  6. 
.     * Logit regression with SEs clustered by DAO
.     logit `y' `user_char', vce(cluster dao)
  7.     eststo `y'_u
  8. 
.     * Logit regression with SEs clustered by DAO
.     logit `y' `user_char' `proposal_char', vce(cluster dao)
  9.     eststo `y'_p_u
 10. 
. }

    Variable |       VIF       1/VIF  
-------------+----------------------
    weighted |      1.50    0.667387
multi_choi~s |      1.38    0.723607
  delegation |      1.24    0.803895
non_whale_~r |      1.01    0.986332
-------------+----------------------
    Mean VIF |      1.28

It

## Delegation Characteristics

In [13]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

foreach y of local wins {
    * Logit regression with SEs clustered by DAO
    logit `y' whale_from_delegation_rate non_whale_from_delegation_rate, vce(cluster dao)
    eststo `y'_f

    logit `y' whale_to_delegation_rate non_whale_to_delegation_rate, vce(cluster dao)
    eststo `y'_t
}

* Export LaTeX table
esttab                                                      ///
    non_whale_victory_vn_f                                  ///
    non_whale_victory_vn_t                                  ///
    non_whale_victory_vp_f                                  ///
    non_whale_victory_vp_t                                  ///
    non_whale_victory_vp_vn_f                               ///
    non_whale_victory_vp_vn_t                               ///
    using "$TABLES/reg_logit_win_delegate.tex", replace     ///
    se star(* 0.10 ** 0.05 *** 0.01)                        ///
    b(%9.3f) se(%9.2f)                                      ///
    label nogaps booktabs nonotes noomitted eqlabels(none)  ///
    mgroups("\${\it Small Win}^{vn}_{i,t}\$"                ///
    "\${\it Small Win}^{vp}_{i,t}\$"                        ///  
    "\${\it Small Win}^{vp,vn}_{i,t}\$",                    ///
    span                                                    ///
    pattern(1 0 0 1 0 0 1 0 0)                                    ///
    prefix(\multicolumn{@span}{c}{)                         ///
    suffix(})                                               ///
    erepeat(\cmidrule(lr){@span}) )                         ///
    substitute("\_" "_")                                    ///
    stats(N r2_p,                                           ///
        fmt(%9.0fc %9.3f)                                   ///
        labels("Observations" "Pseudo R²"))


. 
. ******************************************************
. * LOGISTIC REGRESSIONS
. ******************************************************
. 
. eststo clear

. 
. foreach y of local wins {
  2.     * Logit regression with SEs clustered by DAO
.     logit `y' whale_from_delegation_rate non_whale_from_delegation_rate, vce(
> cluster dao)
  3.     eststo `y'_f
  4. 
.     logit `y' whale_to_delegation_rate non_whale_to_delegation_rate, vce(clus
> ter dao)
  5.     eststo `y'_t
  6. }

Iteration 0:  Log pseudolikelihood = -367.03878  
Iteration 1:  Log pseudolikelihood = -277.46451  
Iteration 2:  Log pseudolikelihood =  -251.4276  
Iteration 3:  Log pseudolikelihood =   -248.838  
Iteration 4:  Log pseudolikelihood = -248.81938  
Iteration 5:  Log pseudolikelihood = -248.81937  

Logistic regression                                     Number of obs =    629
                                                        Wald chi2(2)  =   3.77
                                                  

## Voting Characteristics

In [14]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

logit non_whale_victory_vn whale_vn_hhi non_whale_vn_hhi 
eststo non_whale_victory_vn_h

logit non_whale_victory_vp whale_vs_hhi non_whale_vs_hhi
eststo non_whale_victory_vp_h

logit non_whale_victory_vp_vn whale_vs_hhi non_whale_vn_hhi 
eststo non_whale_victory_vp_vn_h


* Export LaTeX table
esttab                                                     ///
    non_whale_victory_vn_h                                 ///
    non_whale_victory_vp_h                                 ///
    non_whale_victory_vp_vn_h                              ///
    using "$TABLES/reg_logit_win_vote.tex", replace    ///
    se star(* 0.10 ** 0.05 *** 0.01)                       ///
    b(%9.3f) se(%9.2f)                                     ///
    label nogaps booktabs nonotes noomitted eqlabels(none) ///
    mtitles("\${\it Small Win}^{vn}_{i,t}\$"               ///
    "\${\it Small Win}^{vp}_{i,t}\$"                       ///  
    "\${\it Small Win}^{vp,vn}_{i,t}\$")                   ///
    substitute("\_" "_")                                   ///
    stats(N r2_p,                                          ///
        fmt(%9.0fc %9.3f)                                  ///
        labels("Observations" "Pseudo R²"))


. 
. ******************************************************
. * LOGISTIC REGRESSIONS
. ******************************************************
. 
. eststo clear

. 
. logit non_whale_victory_vn whale_vn_hhi non_whale_vn_hhi 

Iteration 0:  Log likelihood = -448.11674  
Iteration 1:  Log likelihood =  -308.5788  
Iteration 2:  Log likelihood =  -292.5276  
Iteration 3:  Log likelihood = -291.37658  
Iteration 4:  Log likelihood = -291.37518  
Iteration 5:  Log likelihood = -291.37518  

Logistic regression                                     Number of obs =    890
                                                        LR chi2(2)    = 313.48
                                                        Prob > chi2   = 0.0000
Log likelihood = -291.37518                             Pseudo R2     = 0.3498

------------------------------------------------------------------------------
non_wha~y_vn | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+--------------------